In [11]:
# A very simple perceptron for classifying american sign language letters
import signdata
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.utils import np_utils
import wandb
from wandb.keras import WandbCallback

In [43]:
print(signdata.letters)

abcdefghiklmnopqrstuvwxy


In [2]:
# logging code
run = wandb.init()
config = run.config
config.team_name = "davros"
config.loss = "categorical_crossentropy"
config.optimizer = "adam"
config.epochs = 10

W&B Run: https://app.wandb.ai/mlclass/asl-july25/runs/qszsopca
Wrap your training loop with `with wandb.monitor():` to display live results.


In [30]:
# load data
(x_test, y_test) = signdata.load_test_data()
(x_train, y_train) = signdata.load_train_data()

In [4]:
img_width = x_test.shape[1]
img_height = x_test.shape[2]

print(img_width, img_height)
input_shape = (img_width, img_height, 1)

28 28


In [5]:
X_test = x_test /255.0
X_train = x_train /255.0

In [22]:
num_classes = y_train.max()+1
print(num_classes)

25


In [44]:
# one hot encode outputs
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [24]:
# Reshape from (num_examples, x, y) to (num_examples, x, y, num_channels)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
print(X_train.shape, X_test.shape)

(27455, 28, 28, 1) (7172, 28, 28, 1)


In [31]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [36]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train,
      epochs=config.epochs,
      verbose=1,
      validation_data=(X_test, y_test),
      callbacks=[WandbCallback(data_type="image", labels=signdata.letters)])

ValueError: Error when checking target: expected dense_6 to have shape (1,) but got array with shape (25,)

In [41]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

7172/7172 [==============================] - 1s 107us/step
Test accuracy: 0.9598438371444507
